### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
# parallel
from functools import partial
from pathlib import Path

In [5]:
import sys
sys.path.insert(0, '../../')

In [8]:
from src.data_sources import *
from src.midi_data import *

In [5]:
version = 'v10'
data_path = Path('data/midi')
version_path = data_path/version
orig_path = version_path/'midi_sources'
metapath = version_path/'metadata'
all_csv = metapath/'midi_sources.csv'
metapath.mkdir(parents=True, exist_ok=True)

In [6]:
def create_paths(dirname):
    "Standardize midi_source paths"
    dir_path = orig_path/dirname
    json_path = metapath/f'{dirname}_metadata.json'
    csv_path = metapath/f'{dirname}_metadata.csv'
    return dir_path, json_path, csv_path

In [26]:
sources = ['hooktheory', 'freemidi', 'midiworld', 'ecomp', 'cprato', 'classic_piano', 'wikifonia', 'lmd_clean', '130k_reddit']

In [8]:
version_path.relative_to(data_path)

PosixPath('v8')

In [9]:
def relative_path(filepath):
    return str(Path(filepath).relative_to(version_path))

### Remove corrupted file - this causes deadlock with music21 processing

In [10]:
corrupted_files = [
    'midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid',
    'ecomp/2004/MORET02.mid',
    'ecomp/2006/Mordvinov9.MID',
    'ecomp/2006/Na06.MID',
    'ecomp/2008/Cui01.MID',
    'ecomp/2008/Cui02.MID',
    'ecomp/2008/Cui03.MID',
    'ecomp/2008/Cui04.MID',
    'ecomp/2008/Cui05.MID',
    'ecomp/2008/Cui06.MID',
    'ecomp/2008/Cui07.MID',
    'ecomp/2008/Cui08.MID',
    'ecomp/2008/Tan01.MID',
    'ecomp/2008/Tan02.MID',
    'ecomp/2008/Tan03.MID',
    'ecomp/2018/KaszoS14.MID',
    'midiworld/named_midi/Rob_Zombie_-_Demonoid_Phenomenon.mid',
    'midiworld/named_midi/Rob_Zombie_-_Demonoid_Phenomenon.mxl',
]
for f in corrupted_files:
    fp = orig_path/f
    if fp.exists(): fp.unlink()

### Hooktheory

In [11]:
ht_path, ht_json, ht_csv = create_paths('hooktheory')
ht_song_list = metapath/'hooktheory_key2info.json'

In [12]:
def song_key(s): return '_'.join(s.parts[-3:-1])

In [13]:
ht_key2info = load_json(ht_song_list)

if ht_key2info is None:
    song_info = list((ht_path/'xml').glob('*/*/*/*.json'))
    ht_key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(ht_key2info, ht_song_list)
len(ht_key2info)

12008

In [14]:
ht_midi_list = list((ht_path/'pianoroll').glob('*/*/*/*_key_cmajor.mid')); 
len(ht_midi_list)

20076

In [15]:
def get_ht_jsonfile(midi_file): # using json instead of midi for metadata
    return str(midi_file.with_suffix('.json')).replace('pianoroll', 'event').replace('_key', '_symbol_key')

In [16]:
def get_hooktheory_attr(fp):
    song_info = ht_key2info[song_key(fp)]
    song_json = json.load(open(get_ht_jsonfile(fp), 'r'))
    metadata = song_json['metadata']
    artist = fp.parts[-3]
    title = fp.parts[-2]
    section = fp.name.split('_')[0]
    
    ht_key = metadata['key']
    ht_mode = metadata['mode']
    if ht_mode is None: ht_mode = 'major'
    ht_offset = keyc_offset(ht_key, ht_mode)
    
    # convert stream here
    return {
        'artist': artist,
        'title': title,
        'midi': relative_path(fp),
        'section': section,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'source': 'hooktheory',
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_offset': ht_offset,
        'ht_time_signature': metadata['beats_in_measure']
    }

In [17]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000]); hook_out

In [18]:
ht_metadata = parse_midi_dir(ht_midi_list, ht_json, base_path=version_path, 
                             meta_func=get_hooktheory_attr)

Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/w/weezer/beverly-hills/intro-and-verse_key_cmajor.mid list index out of range


Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/y/ylvis/the-fox---what-does-the-fox-say/verse-and-pre-chorus_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/j/jay-z/so-ambitious-feat-pharrel-williams/intro_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/j/johnny-cash/folsom-prison-blues/verse_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/j/jack-johnson/do-you-remember/intro-and-verse_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/j/justin-moore/if-heaven-wasnt-so-far-awat/intro-and-verse_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/j/john-c

Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/m/michiru-yamane/gaze-up-at-the-darkness/instrumental_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/m/muse/muscle-museum/intro_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/q/queen/bohemian-rhapsody/pre-outro_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/q/queen/bohemian-rhapsody/outro_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/q/queen/bohemian-rhapsody/instrumental_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/a/adventure-club/wonder/outro_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hooktheory/pianoroll/a/adventure-club/wonder/bridge_key_cmajor.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/hoo

#### Save song_list

In [19]:
ht_metadata = load_json(ht_json)
arr2csv(ht_metadata.values(), ht_csv)
df = pd.read_csv(ht_csv); df.head()

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,source,ht_offset,ht_bpm,artist,ht_key,ht_time_signature,bpm,section,parts,genres
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36.0,...,hooktheory,0,85,wayne-sharpe,C,3,85.0,intro,"intro,chorus",NaN
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32.0,...,hooktheory,0,128,wayne-sharpe,C,4,128.0,chorus,"intro,chorus",NaN
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16.0,...,hooktheory,0,96,what-a-day,C,4,96.0,chorus,chorus,Jazz
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,hooktheory,0,128,weezer,C,4,NaN,intro-and-verse,intro-and-verse,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16.0,...,hooktheory,0,93,weezer,C,4,93.0,chorus,chorus,Rock


## FreeMidi

In [21]:
fm_path, _, fm_csv = create_paths('freemidi')
fm_dance_path = metapath/f'freemidi_dance_metadata.json'
fm_pop_path = metapath/f'freemidi_pop_metadata.json'
list(fm_path.glob('*'))

[PosixPath('data/midi/v8/midi_sources/freemidi/genre-disco'),
 PosixPath('data/midi/v8/midi_sources/freemidi/genre-pop'),
 PosixPath('data/midi/v8/midi_sources/freemidi/genre-dance-eletric'),
 PosixPath('data/midi/v8/midi_sources/freemidi/genre-punk'),
 PosixPath('data/midi/v8/midi_sources/freemidi/genre-hip-hop-rap'),
 PosixPath('data/midi/v8/midi_sources/freemidi/genre-rock')]

In [22]:
def parse_freemidi_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': genres,
        'source': source
    }

In [23]:
d_parse_func = partial(parse_freemidi_songs, genres='dance', source='freemidi')
dir_path = fm_path/'genre-dance-eletric'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_dance_list = parse_midi_dir(file_list, fm_dance_path,
                               base_path=version_path, meta_func=d_parse_func)

In [24]:
p_parse_func = partial(parse_freemidi_songs, genres='pop', source='freemidi')
dir_path = fm_path/'genre-pop'
file_list = get_files(dir_path, extensions=['.mid'], recurse=True)
fm_pop_list = parse_midi_dir(file_list, fm_pop_path,
                             base_path=version_path, meta_func=p_parse_func)

In [25]:
fmd = load_json(fm_dance_path)
fmp = load_json(fm_pop_path)
fm_all = list(fmd.values())+list(fmp.values())
arr2csv(fm_all, fm_csv)
df = pd.read_csv(fm_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,quarter_length
0,dance,NaN,freemidi,midi_sources/freemidi/genre-dance-eletric/Glor...,I'm Not Givin' You Up,NaN,NaN,NaN,NaN,Gloria Estefan,NaN,NaN
1,dance,4/4,freemidi,midi_sources/freemidi/genre-dance-eletric/Radi...,Fitter Happier,"Piano,Piano",G minor,76.0,85.263158,Radiohead,2.0,108.0
2,dance,NaN,freemidi,midi_sources/freemidi/genre-dance-eletric/Tune...,Bounce,NaN,F major,142.0,28.732394,Tune Up,-5.0,68.0
3,dance,4/4,freemidi,midi_sources/freemidi/genre-dance-eletric/Daft...,The Grid,"StringInstrument,StringInstrument,StringInstru...",C# minor,102.0,40.000000,Daft Punk,-4.0,68.0
4,dance,4/4,freemidi,midi_sources/freemidi/genre-dance-eletric/Prod...,Action Radar,Piano,A major,147.0,91.632653,Prodigy,3.0,224.5


### Gather Lakh Midi Dataset

In [26]:
lmd_path, lmd_json, lmd_csv = create_paths('lmd_clean')

In [27]:
def parse_lmd_songs(fp):
    artist = fp.parts[-2]
    title = fp.parts[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': 'pop,inferred',
        'source': 'lmd'
    }

In [28]:
file_list = get_files(lmd_path, extensions=['.mid'], recurse=True)
lmd_md = parse_midi_dir(file_list, lmd_json, base_path=version_path, meta_func=parse_lmd_songs)

Midi Exeption: data/midi/v8/midi_sources/lmd_clean/The Police/So Lonely.1.mid ord() expected string of length 1, but int found
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Phish/Taste.mid badly formated midi bytes, got: b'CAKEWALK\x1a\x00\x03\n\x02\x00\x00\x00x\x00\x03Y'
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Jennifer Lopez/If You Had My Love.mid badly formated midi bytes, got: b'RIFFN\x07\x01\x00RMIDdataA\x07\x01\x00'
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Alice in Chains/Sludge Factory.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Sarah McLachlan/Adia.1.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Yanni/Secret Vows.1.mid index out of range
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Yanni/The Rain Must Fall.1.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Yanni/Aria.1.mid index out 

Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/(New Wave) Polly.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/All Apologies.1.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/Drain You.mid cannot handle ticks per frame: 77
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/Hairspray Queen.1.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/Dive.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/tourette's.1.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/Radio Friendly Unit Shifter.1.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/lmd_clean/Nirvana/Sliver.2.mid badly formed midi string: missing leading MTrk
Midi Exeption: 

In [29]:
lmd = load_json(lmd_json)
arr2csv(lmd.values(), lmd_csv)
df = pd.read_csv(lmd_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,quarter_length
0,"pop,inferred",4/4,lmd,midi_sources/lmd_clean/Moby/J Breas.mid,J Breas.mid,"Piano,StringInstrument",E minor,100.0,142.200000,Moby,5.0,237.0
1,"pop,inferred",4/4,lmd,midi_sources/lmd_clean/Moby/The Blue Terror of...,The Blue Terror of Lawns.mid,Electric Guitar,A minor,86.0,199.736390,Moby,0.0,1699/6
2,"pop,inferred",4/4,lmd,midi_sources/lmd_clean/Joseph Kosma/Autumn Lea...,Autumn Leaves.mid,"Vibraphone,Acoustic Bass,Oboe",E minor,96.0,151.910112,Joseph Kosma,5.0,231.75
3,"pop,inferred",4/4,lmd,midi_sources/lmd_clean/Anne Murray/You Needed ...,You Needed Me.mid,"Guitar,Horn,Flute",B- major,65.0,206.769231,Anne Murray,2.0,224.0
4,"pop,inferred",4/4,lmd,midi_sources/lmd_clean/Hanson/Weird.mid,Weird.mid,"Piano,Piano,Acoustic Bass,Piano,Piano,Piano",B- major,68.0,NaN,Hanson,2.0,272.0


### Gather 130k Reddit

In [30]:
reddit_path, reddit_json, reddit_csv = create_paths('130k_reddit')

In [31]:
def parse_reddit_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    if len(name) == 1:
        artist = fp.parts[-1]
        title = name[0]
    else:
        artist = name[0]
        title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': 'anything,inferred',
        'source': 'reddit'
    }

In [32]:
file_list = get_files(reddit_path, extensions=['.mid'], recurse=True)
file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 400] # over 200mb takes crazy long to analyze
reddit_md = parse_midi_dir(file_list, reddit_json, base_path=version_path, meta_func=parse_reddit_songs)

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/eagles_witchywoman.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/enquantohouversol.mid badly formated midi bytes, got: b'RIFF\xd9.\x06\x00WAVEfmt \x1e\x00\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/et2.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/everybody03.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/every_little_thing.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/EddieCantorMedley.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/entertainer05.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/E/E/earthbnd.mid badl

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/ocean04.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/ordinary1.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/octbass5.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/octwah.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/only_happy_when_it_rains.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/olympic_lillihammer94.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/onlyth.mid badly formated midi bytes, got: b'<HTML>\n<HEAD>\n      '
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/O/O/one04.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_redd

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/P/P/PerfectNanny.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/newyork03.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/newstand.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana - Sliver.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_-_Polly.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_-_Pennyroyal_Tea.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/newstuff.mid badly formated midi bytes, got: b'RIFF\x8a\x05\x00\x00RMIDdata}\x05\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/NEW_YORK.MID list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_-_

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_-_Dumb.mid index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/name3.mid badly formated midi bytes, got: b''
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_-_Lounge_Act.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_Downer.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_-_Mexican_Seafood.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/nylonpad.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/No_woman.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/Nirvana_Dive.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/N/N/neversay.mid badly

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/bussabasiengtien.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/bones2.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/blue_wah.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/BRIT_V3.MID badly formated midi bytes, got: b'MTM\x10   BRITTAIN BASH'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/bjean.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/border.mid badly formated midi bytes, got: b'<HTML><HEAD>\r\n<TITLE'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/Brennan Heart - Feel U Here (Original Mix).mid badly formated midi bytes, got: b'RIFF\xe4\x10\x00\x00RMIDdata\xab\x10\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/B/B/boystomen.mid badly formated midi bytes

...

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/SYNTHS.MID list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/scifidouble.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/SeanPaul_-_SheDoesntMind__PeakBeatz_20130130210127.mid Cannot set partition by 144 (144/1180591620717411303424)
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/Sean Paul - She Doesn't Mind.mid Cannot set partition by 144 (144/1180591620717411303424)
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/shongo.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/soundsil.mid badly formated midi bytes, got: b'<TITLE>Not Found</TI'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/smworldg.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/subdivs.mid index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_red

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/soulrefrain.mid badly formated midi bytes, got: b''
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/sisteract.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/suna.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/S/S/stairway03.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/W/W/windbeneathmywings03.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/W/W/wwfmidibulldog.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/W/W/we_will_rock_you.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/W/W/WC2_13.MID ord() expected string of length 1, but int found
Midi Exeption: data/

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/THE BEATLES.Uncle Albert K.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/thrill2.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/teenspirit.mid badly formated midi bytes, got: b'<html>\n<head>\n      '
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/theroofisleaking.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/this_boy.mid badly formated midi bytes, got: b'kc\x98\xefM\xf69\xc1\x9b}\x8cv\xfb\x19\xbb\xf9\xb5\xc1\xfdR'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/tryme.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/turska2.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/TurnTurnTurn.mid index out of range
Midi Exeption: data/midi/v8/midi_sources/13

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/Tetris Attack - Froggy's Theme.mid index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/thedance04.mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/Thunder.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/track04.mid badly formated midi bytes, got: b'RIFF\xd2\x93\x01\x00WAVEfmt 2\x00\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/two_of_us.mid badly formated midi bytes, got: b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00\x96\x00\x96\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/that_why_you_go.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/thebulls.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/T/T/The Legend of Zelda Oracle of Ages  Orac

Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/rockymthigh.mid badly formated midi bytes, got: b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/Riva - Run Away.mid badly formated midi bytes, got: b'RIFF\x90\x12\x00\x00RMIDdatag\x12\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/Ragnarok Online - Theme of Juno (Guitar Movement).mid list index out of range
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/READTHIS.MID badly formated midi bytes, got: b'\r\nThis sequence perf'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/Ron van den Beuken presents Clokx - Overdrive (Ron van den Beuken Mix).mid badly formated midi bytes, got: b'file does not exist'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/ridelike1.mid badly formated midi bytes, got: b'vti_encoding:SR|utf8'
Midi Exeption: data/midi/v8/midi_sources/130k_reddit/R/R/Rank 1 - It's Up To You

In [33]:
reddit = load_json(reddit_json)
arr2csv(reddit.values(), reddit_csv)
df = pd.read_csv(reddit_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,quarter_length
0,"anything,inferred",4/4,reddit,midi_sources/130k_reddit/Jazz_www.thejazzpage....,124Telephone,NaN,E- major,80.0,9.750000,124Telephone.mid,-3.0,13.0
1,"anything,inferred",4/4,reddit,midi_sources/130k_reddit/Jazz_www.thejazzpage....,037SlapBass2,NaN,C major,78.0,6.730769,037SlapBass2.mid,0.0,8.75
2,"anything,inferred",4/4,reddit,midi_sources/130k_reddit/Jazz_www.thejazzpage....,004ElPiano1,NaN,D minor,120.0,7.000000,004ElPiano1.mid,-5.0,14.0
3,"anything,inferred",4/4,reddit,midi_sources/130k_reddit/Jazz_www.thejazzpage....,111Shanai,"Shehnai,Shehnai",D minor,110.0,6.409091,111Shanai.mid,-5.0,11.75
4,"anything,inferred",4/4,reddit,midi_sources/130k_reddit/Jazz_www.thejazzpage....,005ElPiano2,NaN,G# minor,120.0,6.250000,005ElPiano2.mid,1.0,12.5


### Gather Cprato

In [34]:
cp_path, cp_json, cp_csv = create_paths('cprato')
list(cp_path.glob('*'))[:5]

[PosixPath('data/midi/v8/midi_sources/cprato/Basto - Again And Again (midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v8/midi_sources/cprato/The Weeknd ft. Lana Del Rey - Stargirl Interlude  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v8/midi_sources/cprato/Two Steps From Hell - Magic of Love  (midi by Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v8/midi_sources/cprato/Bermuda Loverz - My Girl (Ladidada) (Rimini Rockaz Radio Edit) (Midi By Carlo Prato) (www.cprato.com).mid'),
 PosixPath('data/midi/v8/midi_sources/cprato/Cascada - Everytime We Touch (Midi By Carlo Prato) (www.cprato.com).mid')]

In [35]:
def parse_cprato_songs(fp, genres=None, source=None):
    name = fp.with_suffix('').name.split(' - ')
    artist = name[0]
    title = name[-1].replace('(midi by Carlo Prato) (www.cprato.com)', '')
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': genres,
        'source': source
    }

In [36]:
cp_meta = partial(parse_cprato_songs, genres='EDM,inferred', source='cprato')
file_list = get_files(cp_path, extensions=['.mid'], recurse=True)
cp_md = parse_midi_dir(file_list, cp_json, 
                       base_path=version_path, meta_func=cp_meta)

In [37]:
cp = load_json(cp_json)
arr2csv(cp.values(), cp_csv)
df = pd.read_csv(cp_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,quarter_length
0,"EDM,inferred",4/4,cprato,midi_sources/cprato/DuMonde - See The Light (...,See The Light,"Piano,Piano,Piano,Piano,Piano,Piano",C# minor,140.0,29.142857,DuMonde,-4,68.0
1,"EDM,inferred",4/4,cprato,midi_sources/cprato/Mura Masa - Suicide Blades...,Suicide Blades,"Piano,Piano,Piano,StringInstrument,StringInstr...",E minor,85.0,48.000000,Mura Masa,5,68.0
2,"EDM,inferred",4/4,cprato,midi_sources/cprato/Avicii - Lonely Together ...,Lonely Together,"Piano,Piano,Piano,Piano,Piano,Piano",F major,103.0,20.970874,Avicii,-5,36.0
3,"EDM,inferred",4/4,cprato,midi_sources/cprato/deadmau5 - Deus Ex Machina...,Deus Ex Machina,"Piano,Piano,Piano,Piano,Piano,Piano",F minor,128.0,69.375000,deadmau5,4,148.0
4,"EDM,inferred",4/4,cprato,midi_sources/cprato/Ephixa & Stephen Walking -...,Matches,"Flute,Flute,Flute,Piano,Piano,Piano,Flute,Flut...",F# minor,108.0,37.777778,Ephixa & Stephen Walking,3,68.0


### Gather MidiWorld

In [38]:
mw_path, mw_json, mw_csv = create_paths('midiworld')

In [39]:
def parse_midiworld_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'midi': relative_path(fp),
        'genres': 'pop,inferred',
        'source': 'midiworld'
    }

In [40]:
file_list = get_files(mw_path/'named_midi', extensions=['.mid'], recurse=True)
mw_md = parse_midi_dir(file_list, mw_json, base_path=version_path, meta_func=parse_midiworld_songs)

Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/Bomfunk_MCs_-_Uprocking_Beats.mid badly formated midi bytes, got: b'RIFF\xb8\xa7\x00\x00RMIDdata\x04\xa7\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/Nirvana_-_Polly.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/Nirvana_-_Pennyroyal_Tea.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/Nirvana_-_Radio_Friendly_Unit_Shifter.mid badly formed midi string: missing leading MTrk
Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/Bomfunk_MCs_-_B-Boys_And_Fly-Girls.mid badly formated midi bytes, got: b'RIFF^\xd8\x00\x00RMIDdata\xa1\xd7\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/Nightwish_-_Feel_For_You.mid badly formated midi bytes, got: b'RIFFr\x9a\x00\x00RMIDdatae\x9a\x00\x00'
Midi Exeption: data/midi/v8/midi_sources/midiworld/named_midi/B

In [41]:
mw = load_json(mw_json)
arr2csv(mw.values(), mw_csv)
df = pd.read_csv(mw_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,quarter_length
0,"pop,inferred",4/4,midiworld,midi_sources/midiworld/named_midi/Kona_-_Druml...,Drumloop,Piano,B minor,176.0,1.193182,Kona,-2.0,3.5
1,"pop,inferred",4/4,midiworld,midi_sources/midiworld/named_midi/Cameron_Lee_...,Cue,Piano,D minor,95.0,2.526316,Cameron Lee Simpson,-5.0,4.0
2,"pop,inferred",4/4,midiworld,midi_sources/midiworld/named_midi/TV_Themes_-_...,Looney Tunes,"Marimba,Acoustic Bass,Trombone,StringInstrumen...",C major,160.0,19.250000,TV Themes,0.0,154/3
3,"pop,inferred",4/4,midiworld,midi_sources/midiworld/named_midi/TV_Themes_-_...,Millenium,"Timpani,Taiko,Violin,Contrabass,Voice",A major,127.0,41.929134,TV Themes,3.0,88.75
4,"pop,inferred",2/4,midiworld,midi_sources/midiworld/named_midi/Kaito_-_Cant...,Cantarella,Piano,C# minor,144.0,67.916667,Kaito,-4.0,163.0


### Gather Wikifonia

In [42]:
wf_path, wf_json, wf_csv = create_paths('wikifonia')

In [43]:
def parse_wikifonia_songs(fp):
    name = fp.with_suffix('').name.replace('_', ' ').split(' - ')
    artist = name[0]
    title = name[-1]
    return {
        'artist': artist.strip(),
        'title': title.strip(),
        'mxl': relative_path(fp),
        'genres': 'pop,inferred',
        'source': 'wikifonia'
    }

In [44]:
# Warning: if you get a deadlock: 
# PosixPath('data/midi/midi_sources/midiworld/named_midi/NITRO_BRO_-_IT_WONT_DIE.mid') is broken
file_list = get_files(wf_path, extensions=['.mxl'], recurse=True)
wf_md = parse_midi_dir(file_list, wf_json, base_path=version_path, meta_func=parse_wikifonia_songs)

Midi Exeption: data/midi/v8/midi_sources/wikifonia/Django Reinhardt - Douce Ambiance.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Charlie Parker - Scrapple From The Apple.mxl Degree not in specified chord: 9


musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 
musicxml.xmlToM21: WARNING: Cannot put in an element with a missing voice tag when no previous voice tag was given.  Assuming voice 1... 


Midi Exeption: data/midi/v8/midi_sources/wikifonia/Bach ? - Four Part Choralforwikifoniachannel04quant.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v8/midi_sources/wikifonia/blah3 - blah.mxl failed to get likely keys for Stream component
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Monty Python - Always look on the bright side of kive.mxl got a negative delta time
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Fraser Smith, Adele Adkins - Set Fire To The Rain.mxl float division by zero
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Spencer, James F. Hanley - Mary Is Talking.mxl failed to get likely keys for Stream component
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Maurice Williams - Stay.mxl unsupported operand type(s) for /: 'int' and 'NoneType'
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Gigi Gryce, Jon Hendricks - Social Call.mxl unsupported operand type(s) for /: 'int' and 'NoneType'
Midi Exeption: data/midi

musicxml.xmlToM21: WARNING: Line <bracket> stop without start


Midi Exeption: data/midi/v8/midi_sources/wikifonia/Joe Harnell - The Lonely Man.mxl Degree not in specified chord: 11
Midi Exeption: data/midi/v8/midi_sources/wikifonia/L. van Beethoven - Himno a la Alegria.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v8/midi_sources/wikifonia/ARR. by Peter Sweeney - WONDERFUL TONIGHT.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Dot145 - Precession of the Equinoxes.mxl Degree not in specified chord: 9
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Ray Henderson, Joseph Widow Young, Sam M. Lewis - Five Foot Two, Eyes Of Blue .mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Clifford Brown - Daahoud.mxl local variable 'lyricLanguage' referenced before assignment
Midi Exeption: data/midi/v8/midi_sources/wikifonia/Gerry Goffin, Michael Masser - Nothing's Gonna Change My Love 

In [45]:
wf = load_json(wf_json)
arr2csv(wf.values(), wf_csv)
df = pd.read_csv(wf_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,mxl,quarter_length
0,"pop,inferred",NaN,wikifonia,NaN,Douce Ambiance,NaN,NaN,NaN,NaN,Django Reinhardt,NaN,midi_sources/wikifonia/Django Reinhardt - Douc...,NaN
1,"pop,inferred",NaN,wikifonia,NaN,Scrapple From The Apple,NaN,NaN,NaN,NaN,Charlie Parker,NaN,midi_sources/wikifonia/Charlie Parker - Scrapp...,NaN
2,"pop,inferred",4/4,wikifonia,midi_sources/from_mxl/wikifonia/Joe Henderson ...,Mamacita,NaN,F minor,NaN,NaN,Joe Henderson,4.0,midi_sources/wikifonia/Joe Henderson - Mamacit...,64.0
3,"pop,inferred",4/4,wikifonia,"midi_sources/from_mxl/wikifonia/Joe Trohman, A...",My Songs Know What You Did In The Dark (Light Em,Piano,D minor,155.0,99.096774,"Joe Trohman, Andy Hurley, Butch Walker, John H...",-5.0,"midi_sources/wikifonia/Joe Trohman, Andy Hurle...",256.0
4,"pop,inferred",4/4,wikifonia,midi_sources/from_mxl/wikifonia/Hungarian folk...,?n az ?jjel nem aludtam egy ?r?t,Grand Piano,C major,NaN,NaN,Hungarian folk song,0.0,midi_sources/wikifonia/Hungarian folk song - ?...,48.0


### Yamaha - piano

In [46]:
ec_path, ec_json, ec_csv = create_paths('ecomp')
ec_song = ec_path/'song_list.json'
list(ec_path.glob('*'))[:5]

[PosixPath('data/midi/v8/midi_sources/ecomp/song_list.json'),
 PosixPath('data/midi/v8/midi_sources/ecomp/2017'),
 PosixPath('data/midi/v8/midi_sources/ecomp/2008'),
 PosixPath('data/midi/v8/midi_sources/ecomp/2006'),
 PosixPath('data/midi/v8/midi_sources/ecomp/2004')]

In [47]:
ec_songs = load_json(ec_path/'song_list.json')

In [48]:
def parse_ecomp_songs(fp):
    song_info = ec_songs[fp.stem]
    return {
        'artist': song_info['artist'],
        'title': song_info['title'],
        'mxl': relative_path(fp),
        'genres': 'classical',
        'source': 'ecomp'
    }

In [49]:
file_list = get_files(ec_path, extensions=['.mxl'], recurse=True)

In [50]:
sorted_files = sorted([fp.stat().st_size/1000 for fp in file_list])[-200:]

In [51]:
file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 400] # over 200mb takes crazy long to analyze
ec_md = parse_midi_dir(file_list, ec_json, base_path=version_path, meta_func=parse_ecomp_songs)

Midi Exeption: data/midi/v8/midi_sources/ecomp/2017/SunY03.mxl got a negative delta time
Midi Exeption: data/midi/v8/midi_sources/ecomp/2006/Tysman02.mxl got a negative delta time
Midi Exeption: data/midi/v8/midi_sources/ecomp/2004/BLINOV01.mxl got a negative delta time
Midi Exeption: data/midi/v8/midi_sources/ecomp/2009/Na03.mxl got a negative delta time
Midi Exeption: data/midi/v8/midi_sources/ecomp/2002/sun01.mxl got a negative delta time


In [52]:
ec = load_json(ec_json)
arr2csv(ec.values(), ec_csv)
df = pd.read_csv(ec_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,mxl,quarter_length
0,classical,3/4,ecomp,midi_sources/from_mxl/ecomp/2017/WangH09.mid,II. Intermezzo in A Minor,"Piano,Piano",A minor,106.0,212.264151,,0,midi_sources/ecomp/2017/WangH09.mxl,375.0
1,classical,4/4,ecomp,midi_sources/from_mxl/ecomp/2017/KaiRuiR06.mid,Etude Op. 10 No. 2 in A Minor,"Piano,Piano",A minor,160.0,79.500000,Frédéric Chopin,0,midi_sources/ecomp/2017/KaiRuiR06.mxl,212.0
2,classical,4/4,ecomp,midi_sources/from_mxl/ecomp/2017/LeeSH12.mid,II. Andantino: Getragen,"Piano,Piano",C major,96.0,277.500000,,0,midi_sources/ecomp/2017/LeeSH12.mxl,444.0
3,classical,4/4,ecomp,midi_sources/from_mxl/ecomp/2017/WangH12.mid,VII. Capriccio in D Minor,"Piano,Piano",A minor,157.0,122.292994,,0,midi_sources/ecomp/2017/WangH12.mxl,320.0
4,classical,4/4,ecomp,midi_sources/from_mxl/ecomp/2017/WangA01.mid,"Prelude and Fugue in E Major, WTC I, BWV 854","Piano,Piano",E major,74.0,149.189189,Johann Sebastian Bach,-4,midi_sources/ecomp/2017/WangA01.mxl,184.0


### Reformatting code

In [53]:
# def reformat_json(out_path):
#     f2m = load_json(out_path)
#     m = {relative_path(k):v for k,v in f2m.items()}
#     save_json(m, out_path)
#     return m

In [54]:
# def reformat_key(k):
#     return k.replace('data/midi/midi_sources_fromxml_v3','midi_sources/from_mxl')

In [55]:
# def reformat_data(d):
#     if 'midi' in d: d['midi'] = reformat_key(d['midi'])
#     if 'mxl' in d: d['mxl'] = d['mxl'].replace('data/midi/midi_sources_v3', 'midi_sources')
#     return d

In [56]:
# f2m = load_json(clc_json)
# m = {reformat_data(v)['mxl']:reformat_data(v) for k,v in f2m.items() if 'mxl' in reformat_data(v)}
# save_json(m, clc_json)

### Classic Piano

In [57]:
clc_path, clc_json, clc_csv = create_paths('classic_piano')
list(clc_path.glob('*'))[:5]

[PosixPath('data/midi/v8/midi_sources/classic_piano/liz_rhap15_format0.mxl'),
 PosixPath('data/midi/v8/midi_sources/classic_piano/ty_september_format0.mxl'),
 PosixPath('data/midi/v8/midi_sources/classic_piano/clementi_opus36_2_2_format0.mid'),
 PosixPath('data/midi/v8/midi_sources/classic_piano/mz_333_2_format0.mid'),
 PosixPath('data/midi/v8/midi_sources/classic_piano/haydn_7_1_format0.mid')]

In [58]:
def parse_classic_songs(fp):
    name = fp.with_suffix('').name.split('_')
    artist = name[0]
    title = ' '.join(name[1:])
    return {
        'artist': artist,
        'title': title,
        'mxl': relative_path(fp),
        'genres': 'classical',
        'source': 'classical_piano'
    }

In [59]:
file_list = get_files(clc_path, extensions=['.mxl'], recurse=True)

In [60]:
file_list = [fp for fp in file_list if fp.stat().st_size/1000 < 350] # over 200mb takes crazy long to analyze

In [61]:
clc_md = parse_midi_dir(file_list, clc_json, base_path=version_path, meta_func=parse_classic_songs)

In [62]:
clc = load_json(clc_json)
arr2csv(clc.values(), clc_csv)
df = pd.read_csv(clc_csv); df.head()

,genres,time_signature,source,midi,title,instruments,inferred_key,bpm,seconds,artist,inferred_offset,mxl,quarter_length
0,classical,6/8,classical_piano,midi_sources/from_mxl/classic_piano/chpn-p11_f...,format0,"Piano,Piano",B major,111.0,29.635549,chpn-p11,1,midi_sources/classic_piano/chpn-p11_format0.mxl,81.0
1,classical,3/4,classical_piano,midi_sources/from_mxl/classic_piano/haydn_8_2_...,8 2 format0,"Piano,Piano",G major,100.0,57.682776,haydn,5,midi_sources/classic_piano/haydn_8_2_format0.mxl,96.0
2,classical,2/4,classical_piano,midi_sources/from_mxl/classic_piano/scn15_3_fo...,3 format0,"Piano,Piano",E minor,115.0,31.903386,scn15,5,midi_sources/classic_piano/scn15_3_format0.mxl,64.0
3,classical,6/8,classical_piano,midi_sources/from_mxl/classic_piano/waldstein_...,2 format0,"Piano,Piano",C major,20.0,245.746646,waldstein,0,midi_sources/classic_piano/waldstein_2_format0...,84.0
4,classical,4/4,classical_piano,midi_sources/from_mxl/classic_piano/chpn-p18_f...,format0,"Piano,Piano",F minor,126.0,41.887744,chpn-p18,4,midi_sources/classic_piano/chpn-p18_format0.mxl,85.0


### Creating CSV

In [27]:
all_csvs = [create_paths(s)[-1] for s in sources]
all_dfs = [pd.read_csv(csv) for csv in all_csvs]

In [28]:
merged_df = pd.concat(all_dfs, sort=False)
merged_df = merged_df.reset_index(drop=True); merged_df

,inferred_offset,song_url,instruments,ht_mode,midi_title,title,seconds,midi,inferred_key,quarter_length,...,ht_offset,ht_bpm,artist,ht_key,ht_time_signature,bpm,section,parts,genres,mxl
0,0.0,https://www.hooktheory.com/theorytab/view/wayn...,Piano,1.0,yu-gi-oh,yu-gi-oh-theme-song,25.411765,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,36,...,0.0,85.0,wayne-sharpe,C,3.0,85.00,intro,"intro,chorus",NaN,NaN
1,0.0,https://www.hooktheory.com/theorytab/view/wayn...,"Piano,Piano",1.0,yu-gi-oh3,yu-gi-oh-theme-song,15.000000,midi_sources/hooktheory/pianoroll/w/wayne-shar...,C major,32,...,0.0,128.0,wayne-sharpe,C,4.0,128.00,chorus,"intro,chorus",NaN,NaN
2,5.0,https://www.hooktheory.com/theorytab/view/what...,"Piano,Piano",1.0,kiefer,kiefer,10.000000,midi_sources/hooktheory/pianoroll/w/what-a-day...,E minor,16,...,0.0,96.0,what-a-day,C,4.0,96.00,chorus,chorus,Jazz,NaN
3,NaN,https://www.hooktheory.com/theorytab/view/weez...,NaN,1.0,My New Song,beverly-hills,NaN,midi_sources/hooktheory/pianoroll/w/weezer/bev...,NaN,NaN,...,0.0,128.0,weezer,C,4.0,NaN,intro-and-verse,intro-and-verse,NaN,NaN
4,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Weezer - Fall Together,fall-together-,10.322581,midi_sources/hooktheory/pianoroll/w/weezer/fal...,A minor,16,...,0.0,93.0,weezer,C,4.0,93.00,chorus,chorus,Rock,NaN
5,-5.0,https://www.hooktheory.com/theorytab/view/wavv...,"Piano,Piano",1.0,afraidofheightsintro,afraid-of-heights,8.135593,midi_sources/hooktheory/pianoroll/w/wavves/afr...,F major,16,...,0.0,118.0,wavves,C,4.0,118.00,intro,"intro,verse,chorus",NaN,NaN
6,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,Buddy Holly,buddy-holly,16.000000,midi_sources/hooktheory/pianoroll/w/weezer/bud...,C major,32,...,0.0,120.0,weezer,C,4.0,120.00,chorus,"verse,pre-chorus,chorus,bridge,solo","Pop,Rock",NaN
7,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,greatest man,the-greatest-man-that-ever-lived,24.615385,midi_sources/hooktheory/pianoroll/w/weezer/the...,C major,32,...,0.0,78.0,weezer,C,4.0,78.00,solo-1,"intro,solo-1",NaN,NaN
8,0.0,https://www.hooktheory.com/theorytab/view/weez...,"Piano,Piano",1.0,falling for you intro,falling-for-you,11.111111,midi_sources/hooktheory/pianoroll/w/weezer/fal...,C major,20,...,0.0,108.0,weezer,C,4.0,108.00,intro,"intro,verse,chorus,solo",NaN,NaN
9,0.0,https://www.hooktheory.com/theorytab/view/wavv...,"Piano,Piano",1.0,afraidverse,afraid-of-heights,17.454545,midi_sources/hooktheory/pianoroll/w/wavves/afr...,C major,32,...,0.0,110.0,wavves,C,4.0,110.00,verse,"intro,verse,chorus",NaN,NaN


In [29]:
[df.shape for df in all_dfs], merged_df.shape

([(20076, 21),
  (5784, 12),
  (4711, 12),
  (2538, 13),
  (314, 12),
  (328, 13),
  (6391, 13),
  (17243, 12),
  (128461, 12)],
 (185846, 22))

In [30]:
merged_df.to_csv(all_csv, index=False)